In [11]:
#This project is confusing. We cannot recreate the R code here as factoring in R is different than pandas categoricals. 
#I dont know which columns are valid and invalid in demo code so I am going with my judgement from all the knowledge I acumulated from the courcework here.

# DELIVERABLE -1


# Context and Fairness Definition

**Context.** The Civilian Complaint Review Board (CCRB) evaluates civilian allegations against NYPD officers and decides whether each allegation is valid or invalid, and this dataset contains many years of those allegation-level decisions along with officer and complainant attributes.

**Fairness definition.** In this project I define fairness as a clear, argued commitment to what we protect, how we measure it, and where we intervene in the workflow. I treat **race** and **gender** as protected because they are legally and socially salient in policing outcomes, and unequal error burdens across these attributes risk compounding historic disadvantage and eroding public trust. My primary concern is two kinds of harm: missed true harms to complainants and unjust false positives for officers. After considering metrics discussed in Mulligan et al. such as **Demographic Parity**, **Equalized Odds**, **Predictive Value Parity**, and **calibration by group**, I adopt an **Equal Opportunity style criterion** within the Equalized Odds family that prioritizes **parity in true positive rates** across protected groups, since the CCRB’s accountability mission is to recognise genuine misconduct equally across groups. I explicitly prefer this over demographic parity, which can be misleading when base rates differ and can force equalisation that ignores case facts, and over predictive parity, which can tolerate unequal sensitivity and leave some groups with systematically lower chances of having true harms recognised. I accept that this choice may yield different positive prediction rates across groups; my goal is to avoid unequal under-detection of genuine misconduct while I monitor **false positive rate** gaps and **calibration** by group as guardrails to protect officers from unfair sanctioning. I evaluate gender alongside race because alleged harms and disciplinary outcomes can vary by gender in policy-relevant ways, and excluding gender could mask unequal burdens. Given New York City’s history of strained police community relations and racial disparities in enforcement, fairness here is not only statistical but also about public trust for complainants, officers, CCRB staff, NYPD leadership, advocacy groups, and the public. In line with Mulligan’s call for shared vocabularies and explicit scope, I will publish decision criteria and thresholds, audit **TPR** and **FPR** by group on a schedule, show calibration by group, and state trade offs up front so stakeholders understand both the measurement and the actions that follow.


# DELIVERABLE -2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import patsy as pt
from statsmodels.stats.proportion import proportion_confint

In [6]:
OFFICERS_PATH   = r"E:\DSC451-ethics\project-2\Civilian_Complaint_Review_Board__Police_Officers_20251027.csv"
COMPLAINTS_PATH = r"E:\DSC451-ethics\project-2\Civilian_Complaint_Review_Board__Complaints_Against_Police_Officers_20251027.csv"
ALLEGATIONS_PATH= r"E:\DSC451-ethics\project-2\Civilian_Complaint_Review_Board__Allegations_Against_Police_Officers_20251027.csv"
PENALTIES_CSV  = r"E:\DSC451-ethics\project-2\Civilian_Complaint_Review_Board__Penalties_20251027.csv"

officers   = pd.read_csv(OFFICERS_PATH,   na_values=["", "NA"], low_memory=False)
complaints = pd.read_csv(COMPLAINTS_PATH, na_values=["", "NA"], low_memory=False)
allegations= pd.read_csv(ALLEGATIONS_PATH,na_values=["", "NA"], low_memory=False)
penalties   = pd.read_csv(PENALTIES_CSV,   na_values=["", "NA"], low_memory=False)

print("Shapes -> officers, complaints, allegations:")
print(officers.shape, complaints.shape, allegations.shape, penalties.shape)

def cols(df): 
    return [c for c in df.columns if "Complaint" in c or "Tax" in c or "Allegation" in c or "Penalty" in c or "Rank" in c][:15]

print("\nKey-like columns:")
print("Officers:", cols(officers))
print("Complaints:", cols(complaints))
print("Allegations:", cols(allegations))
print("Penalties:", cols(penalties))

Shapes -> officers, complaints, allegations:
(95322, 14) (136395, 14) (411977, 18) (13980, 13)

Key-like columns:
Officers: ['Tax ID', 'Current Rank Abbreviation', 'Current Rank', 'Total Complaints', 'Total Substantiated Complaints']
Complaints: ['Complaint Id', 'CCRB Complaint Disposition']
Allegations: ['Complaint Id', 'Complaint Officer Number', 'Tax ID', 'Officer Rank Abbreviation At Incident', 'Officer Rank At Incident', 'Allegation Record Identity', 'Allegation', 'CCRB Allegation Disposition', 'NYPD Allegation Disposition']
Penalties: ['Complaint Id', 'Tax ID', 'Non-APU NYPD Penalty Report Date', 'APU CCRB Trial Recommended Penalty', 'APU Trial Commissioner Recommended Penalty', 'APU Plea Agreed Penalty', 'NYPD Officer Penalty']


Just reading all files and wrote and extra line of code in which it considers all NA or empty strings as missing.

In [7]:
#checking for missing values
def missing_report(df: pd.DataFrame, name: str, top=20):
    rep = (
        pd.DataFrame({
            "column": df.columns,
            "n_missing": df.isna().sum().values,
            "pct_missing": (df.isna().mean().values * 100).round(2)
        })
        .sort_values(["n_missing","pct_missing"], ascending=False)
        .reset_index(drop=True)
    )
    rows_with_any_missing = int(df.isna().any(axis=1).sum())
    total_cells = df.shape[0] * df.shape[1]
    total_missing = int(df.isna().sum().sum())
    print(f"\n=== Missing values: {name} (rows={len(df)}, cols={df.shape[1]}) ===")
    print(f"Total missing cells: {total_missing:,} out of {total_cells:,}")
    print(f"Rows with any missing: {rows_with_any_missing:,}")
    print(rep.head(top).to_string(index=False))

# Run for each table you loaded in Step 1
missing_report(officers,   "Officers")
missing_report(complaints, "Complaints")
missing_report(allegations,"Allegations")
missing_report(penalties,  "Penalties")


=== Missing values: Officers (rows=95322, cols=14) ===
Total missing cells: 81 out of 1,334,508
Rows with any missing: 81
                         column  n_missing  pct_missing
                      Shield No         58         0.06
                 Officer Gender         23         0.02
                     As Of Date          0         0.00
                         Tax ID          0         0.00
Active Per Last Reported Status          0         0.00
      Last Reported Active Date          0         0.00
             Officer First Name          0         0.00
              Officer Last Name          0         0.00
                   Officer Race          0         0.00
      Current Rank Abbreviation          0         0.00
                   Current Rank          0         0.00
                Current Command          0         0.00
               Total Complaints          0         0.00
 Total Substantiated Complaints          0         0.00

=== Missing values: Complaints (rows

In [8]:
DISP = "CCRB Complaint Disposition"
KEY  = "Complaint Id"

In [9]:
if DISP not in allegations.columns:
    allegations = allegations.merge(
        complaints[[KEY, DISP]],
        on=KEY,
        how="left",
        validate="many_to_one"   # each complaint has one disposition, many allegations reference it
    )

# Sanity: confirm the column is present and inspect missingness
assert DISP in allegations.columns, "Disposition column not found on Allegations after merge."

n_miss = allegations[DISP].isna().sum()
print(f"Allegations now has '{DISP}'. Missing values: {n_miss:,} of {len(allegations):,}")

# Peek at the most common labels so we know what we are about to map
print("\nTop complaint dispositions now on Allegations:")
print(allegations[DISP].value_counts(dropna=False).head(20))

Allegations now has 'CCRB Complaint Disposition'. Missing values: 0 of 411,977

Top complaint dispositions now on Allegations:
CCRB Complaint Disposition
Complainant Uncooperative                   93737
Unsubstantiated                             86396
Unfounded                                   34139
Complaint Withdrawn                         33126
Substantiated (Charges)                     29562
Complainant Unavailable                     26255
Exonerated                                  19743
Alleged Victim Uncooperative                13219
Closed - Pending Litigation                 11335
Substantiated (Command Discipline A)        11221
Officer(s) Unidentified                      9029
Substantiated (Command Discipline B)         8642
Unable to Determine                          8250
Substantiated (Formalized Training)          6311
Alleged Victim Unavailable                   4654
Substantiated (Command Discipline)           3303
Within NYPD Guidelines                       2

In [10]:
def _norm(s):
    return "" if pd.isna(s) else str(s).strip()

# Valid: any "Substantiated (...)" label
def is_valid(s):
    return _norm(s).startswith("Substantiated")

# Policy choice for main analysis:
# Count "Unable to Determine" as INVALID (we will do a sensitivity later excluding it)
INVALID_SET = {
    "Unfounded",
    "Exonerated",
    "Within NYPD Guidelines",
    "Unsubstantiated",
    "Unable to Determine"
}
def is_invalid(s):
    return _norm(s) in INVALID_SET

# Process closures to EXCLUDE from decided set
EXCLUDED_PREFIXES = {
    "Complainant Uncooperative",
    "Alleged Victim Uncooperative",
    "Complainant Unavailable",
    "Alleged Victim Unavailable",
    "Complaint Withdrawn",
    "Officer(s) Unidentified",
    "Victim Unidentified",
    "Mediation Attempted",
    "Mediated",
    "Closed - Pending Litigation",
    "OMB PEG Directive Closure",
    "SRAD Closure",
    "Administratively Closed",
    "Miscellaneous",
    "Miscellaneous - Subject Retired",
    "Miscellaneous - Subject Resigned",
    "Witness Uncooperative",
    "Witness Unavailable",
}
def is_excluded(s):
    s = _norm(s)
    return any(s.startswith(p) for p in EXCLUDED_PREFIXES)

allegations["ccrb_valid"]   = allegations[DISP].map(is_valid)
allegations["ccrb_invalid"] = allegations[DISP].map(is_invalid)
allegations["ccrb_exclude"] = allegations[DISP].map(is_excluded)

# Sanity bucket to confirm mapping coverage
bucket = np.where(allegations["ccrb_valid"],   "VALID",
         np.where(allegations["ccrb_invalid"], "INVALID",
         np.where(allegations["ccrb_exclude"], "EXCLUDED", "UNCAPTURED")))

print(pd.Series(bucket, name="bucket").value_counts().sort_index())

# If anything remains UNCATURED, print top labels so we can improve rules
uncaptured = allegations.loc[(bucket=="UNCAPTURED"), DISP].value_counts()
if len(uncaptured):
    print("\nUNCAPTURED labels (review mapping):")
    print(uncaptured.head(20))
else:
    print("\nAll dispositions captured by the mapping.")


bucket
EXCLUDED    198240
INVALID     151468
VALID        62269
Name: count, dtype: int64

All dispositions captured by the mapping.


In [12]:
decided = allegations[(allegations["ccrb_valid"]) | (allegations["ccrb_invalid"])].copy()

# Binary outcome: 1 = substantiated, 0 = not substantiated
decided["outcome_bin"] = decided["ccrb_valid"].astype(int)

# Report
n_decided = len(decided)
substantiation_rate = decided["outcome_bin"].mean()

print(f"Decided shape: {decided.shape}")
print(f"Overall substantiation rate (decided): {substantiation_rate:.4f}  ({substantiation_rate*100:.2f}%)")
print(decided["CCRB Complaint Disposition"].value_counts().head(10))

Decided shape: (213737, 23)
Overall substantiation rate (decided): 0.2913  (29.13%)
CCRB Complaint Disposition
Unsubstantiated                         86396
Unfounded                               34139
Substantiated (Charges)                 29562
Exonerated                              19743
Substantiated (Command Discipline A)    11221
Substantiated (Command Discipline B)     8642
Unable to Determine                      8250
Substantiated (Formalized Training)      6311
Substantiated (Command Discipline)       3303
Within NYPD Guidelines                   2940
Name: count, dtype: int64


In [ ]:
#as I chose equal opportunity as my fairness metric
# Attach officer race/gender to the decided set
decided = decided.merge(
    officers[["Tax ID", "Officer Race", "Officer Gender"]],
    on="Tax ID", how="left"
).rename(columns={"Officer Race":"officer_race", "Officer Gender":"officer_gender"})

coverage = {
    "decided_total": len(decided),
    "with_TaxID": decided["Tax ID"].notna().sum(),
    "with_officer_race": decided["officer_race"].notna().sum(),
    "with_officer_gender": decided["officer_gender"].notna().sum(),
}
print("Coverage:", coverage)



def rates_with_ci(frame: pd.DataFrame, group_col: str) -> pd.DataFrame:
    t = (frame.groupby(group_col, dropna=False)["outcome_bin"]
         .agg(rate="mean", n="count")
         .sort_values("n", ascending=False)
         .reset_index())
    # Wilson 95% CI
    ci_low, ci_high = proportion_confint(
        count=(t["rate"]*t["n"]).round().astype(int),
        nobs=t["n"], method="wilson"
    )
    t["ci_low"]  = ci_low
    t["ci_high"] = ci_high
    return t

# Include "Unknown" buckets for transparency, then compute known-only views
decided["officer_race_f"]   = decided["officer_race"].fillna("Unknown")
decided["officer_gender_f"] = decided["officer_gender"].fillna("Unknown")

by_race_all   = rates_with_ci(decided, "officer_race_f")
by_gender_all = rates_with_ci(decided, "officer_gender_f")

print("\nBy officer race (includes 'Unknown'):")
print(by_race_all)

print("\nBy officer gender (includes 'Unknown'):")
print(by_gender_all)

# Known-only versions 
by_race_known   = rates_with_ci(decided[decided["officer_race_f"]   != "Unknown"], "officer_race_f")
by_gender_known = rates_with_ci(decided[decided["officer_gender_f"] != "Unknown"], "officer_gender_f")

print("\nBy officer race (known only):")
print(by_race_known)

print("\nBy officer gender (known only):")
print(by_gender_known)


Coverage: {'decided_total': 213737, 'with_TaxID': 182778, 'with_officer_race': 182764, 'with_officer_gender': 182693}

By officer race (includes 'Unknown'):
    officer_race_f      rate      n    ci_low   ci_high
0            White  0.294861  95350  0.291975  0.297763
1         Hispanic  0.350442  51201  0.346321  0.354586
2          Unknown  0.144255  30973  0.140386  0.148212
3            Black  0.304178  26810  0.298699  0.309712
4            Asian  0.384440   9216  0.374558  0.394418
5  American Indian  0.240642    187  0.184989  0.306735

By officer gender (includes 'Unknown'):
  officer_gender_f      rate       n    ci_low   ci_high
0             Male  0.316396  166933  0.314170  0.318632
1          Unknown  0.144537   31044  0.140669  0.148492
2           Female  0.314867   15686  0.307644  0.322180
3     TGNC / Other  0.351351      74  0.252382  0.464992

By officer race (known only):
    officer_race_f      rate      n    ci_low   ci_high
0            White  0.294861  95350  0

In [14]:
def disparity_vs_ref(rate_table: pd.DataFrame, group_col: str) -> pd.DataFrame:
    # Identify reference as the largest-N group
    ref = rate_table.loc[rate_table["n"].idxmax()]
    ref_rate  = ref["rate"]
    ref_group = ref[group_col]
    out = rate_table.copy()
    out["ref_group"] = ref_group
    out["risk_diff_vs_ref"]  = out["rate"] - ref_rate
    out["risk_ratio_vs_ref"] = np.where(ref_rate > 0, out["rate"] / ref_rate, np.nan)
    # Order: biggest samples first, then largest absolute difference
    return out.sort_values(["n", "risk_diff_vs_ref"], ascending=[False, True])

disp_race   = disparity_vs_ref(by_race_known,   "officer_race_f")
disp_gender = disparity_vs_ref(by_gender_known, "officer_gender_f")

print("\nDisparities vs reference (officer race):")
print(disp_race[["officer_race_f","n","rate","ref_group","risk_diff_vs_ref","risk_ratio_vs_ref"]])

print("\nDisparities vs reference (officer gender):")
print(disp_gender[["officer_gender_f","n","rate","ref_group","risk_diff_vs_ref","risk_ratio_vs_ref"]])

# Optional: simple EO flags if absolute difference > 3 percentage points
EO_THRESHOLD = 0.03  # 3 pp
def eo_flags(rate_table: pd.DataFrame, label: str):
    ref = rate_table.loc[rate_table["n"].idxmax(), "rate"]
    flagged = rate_table.assign(
        abs_diff_vs_ref = (rate_table["rate"] - ref).abs(),
        eo_flag = np.where((rate_table["rate"] - ref).abs() > EO_THRESHOLD, "FLAG", "OK")
    )
    print(f"\nEO flags for {label} (>|{EO_THRESHOLD*100:.0f}| pp):")
    print(flagged[[rate_table.columns[0],"n","rate","abs_diff_vs_ref","eo_flag"]])

eo_flags(by_race_known,   "officer_race")
eo_flags(by_gender_known, "officer_gender")


Disparities vs reference (officer race):
    officer_race_f      n      rate ref_group  risk_diff_vs_ref  \
0            White  95350  0.294861     White          0.000000   
1         Hispanic  51201  0.350442     White          0.055581   
2            Black  26810  0.304178     White          0.009317   
3            Asian   9216  0.384440     White          0.089579   
4  American Indian    187  0.240642     White         -0.054219   

   risk_ratio_vs_ref  
0           1.000000  
1           1.188500  
2           1.031596  
3           1.303801  
4           0.816119  

Disparities vs reference (officer gender):
  officer_gender_f       n      rate ref_group  risk_diff_vs_ref  \
0             Male  166933  0.316396      Male          0.000000   
1           Female   15686  0.314867      Male         -0.001530   
2     TGNC / Other      74  0.351351      Male          0.034955   

   risk_ratio_vs_ref  
0           1.000000  
1           0.995165  
2           1.110478  

EO flag

In [15]:


#Ensure context variables from Complaints are present
KEY = "Complaint Id"
need_cols = ["Borough Of Incident Occurrence", "Location Type Of Incident", "Incident Date"]
missing = [c for c in need_cols if c not in decided.columns]
if missing:
    decided = decided.merge(complaints[[KEY] + need_cols], on=KEY, how="left")

#  Year
decided["year"] = pd.to_datetime(decided["Incident Date"], errors="coerce").dt.year

#  Bring FADO Type if not already present
if "FADO Type" not in decided.columns and "FADO Type" in allegations.columns:
    # this introduces some noise. A stricter approach uses allegation-level keys.
    decided = decided.merge(allegations[[KEY, "FADO Type"]].drop_duplicates(), on=KEY, how="left")

#  Keep rows with known protected attributes (to interpret coefficients cleanly)
dfm = decided.copy()
dfm = dfm[dfm["officer_race"].notna() & dfm["officer_gender"].notna()].copy()

# Build formula with categorical encodings
terms = [
    "C(officer_race)",
    "C(officer_gender)",
    "C(Q('Borough Of Incident Occurrence'))",
    "C(Q('Location Type Of Incident'))",
    "C(year)"
]
if "FADO Type" in dfm.columns:
    terms.append("C(Q('FADO Type'))")

formula = "outcome_bin ~ " + " + ".join(terms)

#  Design matrices and fit
y, X = pt.dmatrices(formula, data=dfm, return_type="dataframe")
logit = sm.Logit(y, X).fit(disp=False)

print("\nAdjusted logistic regression: odds on substantiation")
print(logit.summary2().tables[1].head(30))  # top rows

#  Handy helper to print exponentiated coefficients (odds ratios) for protected terms
def print_or(prefix="C(officer_race)"):
    coefs = logit.params.filter(like=prefix)
    conf  = logit.conf_int().loc[coefs.index]
    OR    = np.exp(coefs)
    OR_lo = np.exp(conf[0])
    OR_hi = np.exp(conf[1])
    out = pd.DataFrame({"OR": OR, "OR_lo": OR_lo, "OR_hi": OR_hi, "p": logit.pvalues.loc[coefs.index]})
    # Sort by absolute log-odds (magnitude)
    print(f"\nOdds ratios for {prefix}:")
    print(out.sort_values("OR"))

print_or("C(officer_race)")
print_or("C(officer_gender)")


c:\Users\DELL\anaconda3\envs\souq_e_commerce_3_11\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "



Adjusted logistic regression: odds on substantiation
                                                        Coef.    Std.Err.  \
Intercept                                          -13.968535  183.565302   
C(officer_race)[T.Asian]                            -0.157357    0.184802   
C(officer_race)[T.Black]                            -0.125729    0.183834   
C(officer_race)[T.Hispanic]                         -0.053577    0.183583   
C(officer_race)[T.White]                            -0.085754    0.183383   
C(officer_gender)[T.Male]                            0.153666    0.020337   
C(officer_gender)[T.TGNC / Other]                    0.215031    0.267978   
C(Q('Borough Of Incident Occurrence'))[T.Brooklyn]  -0.204044    0.015000   
C(Q('Borough Of Incident Occurrence'))[T.Manhat...  -0.303968    0.016825   
C(Q('Borough Of Incident Occurrence'))[T.Outsid...   0.084917    0.144278   
C(Q('Borough Of Incident Occurrence'))[T.Queens]    -0.320044    0.018885   
C(Q('Borough Of Incide

In [16]:
def rates_with_ci(frame: pd.DataFrame, group_col: str) -> pd.DataFrame:
    t = (frame.groupby(group_col, dropna=False)["outcome_bin"]
         .agg(rate="mean", n="count")
         .sort_values("n", ascending=False)
         .reset_index())
    ci_low, ci_high = proportion_confint(
        count=(t["rate"]*t["n"]).round().astype(int),
        nobs=t["n"], method="wilson"
    )
    t["ci_low"]  = ci_low
    t["ci_high"] = ci_high
    return t

# Ensure FADO Type present
if "FADO Type" not in decided.columns and "FADO Type" in allegations.columns:
    decided = decided.merge(
        allegations[["Complaint Id","FADO Type"]].drop_duplicates(),
        on="Complaint Id", how="left"
    )

# Period buckets if not already present
if "year" not in decided.columns:
    decided["year"] = pd.to_datetime(decided["Incident Date"], errors="coerce").dt.year
decided["period"] = pd.cut(decided["year"],
                           bins=[1999,2005,2010,2015,2020,2026],
                           labels=["2000-05","2006-10","2011-15","2016-20","2021-25"])

# 9a) By FADO type, race differences
if "FADO Type" in decided.columns:
    top_fado = decided["FADO Type"].value_counts(dropna=True).index[:4]
    for f in top_fado:
        sub = decided[(decided["FADO Type"] == f) & (decided["officer_race"].notna())].copy()
        sub["officer_race_f"] = sub["officer_race"]
        tab = rates_with_ci(sub, "officer_race_f")
        print(f"\nBy officer race within FADO = {f}")
        print(tab)

# 9b) By time period, race differences
for p in decided["period"].dropna().unique():
    sub = decided[(decided["period"] == p) & (decided["officer_race"].notna())].copy()
    sub["officer_race_f"] = sub["officer_race"]
    tab = rates_with_ci(sub, "officer_race_f")
    print(f"\nBy officer race within period = {p}")
    print(tab)


By officer race within FADO = Abuse of Authority
    officer_race_f      rate      n    ci_low   ci_high
0            White  0.342474  53350  0.338459  0.346512
1         Hispanic  0.406062  29163  0.400439  0.411711
2            Black  0.355903  14917  0.348257  0.363622
3            Asian  0.431266   5354  0.418054  0.444577
4  American Indian  0.270270    111  0.196369  0.359540

By officer race within FADO = Force
    officer_race_f      rate      n    ci_low   ci_high
0            White  0.210734  25136  0.205736  0.215819
1         Hispanic  0.257708  13589  0.250424  0.265130
2            Black  0.224953   7406  0.215587  0.234604
3            Asian  0.299528   2544  0.282042  0.317619
4  American Indian  0.209302     43  0.114228  0.352056

By officer race within FADO = Discourtesy
    officer_race_f      rate      n    ci_low   ci_high
0            White  0.269025  14218  0.261799  0.276376
1         Hispanic  0.301247   7137  0.290712  0.311996
2            Black  0.256059  

In [17]:
def _norm(s): return "" if pd.isna(s) else str(s).strip()
def is_utd(s): return _norm(s) == "Unable to Determine"

mask_valid   = allegations["ccrb_valid"]
mask_invalid = allegations["ccrb_invalid"] & ~allegations["CCRB Complaint Disposition"].map(is_utd)

decided_exclUTD = allegations[mask_valid | mask_invalid].copy()
decided_exclUTD["outcome_bin"] = decided_exclUTD["ccrb_valid"].astype(int)

print("\n[Sensitivity] Decided excluding UTD shape:", decided_exclUTD.shape)
print("[Sensitivity] Overall substantiation rate:", round(decided_exclUTD["outcome_bin"].mean(), 4))


[Sensitivity] Decided excluding UTD shape: (205487, 23)
[Sensitivity] Overall substantiation rate: 0.303


In [19]:
overall_rate = decided["outcome_bin"].mean()*100
race_tab = by_race_known.sort_values("n", ascending=False)
gender_tab = by_gender_known.sort_values("n", ascending=False)

print("\n--- Summary  ---")
print(f"Among decided allegations (n={len(decided):,}), the substantiation rate is {overall_rate:.1f} percent.")
print("Unadjusted, substantiation rates differ by officer race (e.g., reference ≈ "
      f"{race_tab.iloc[0]['officer_race_f']}: {race_tab.iloc[0]['rate']*100:.1f} percent; "
      f"{race_tab.iloc[1]['officer_race_f']}: {race_tab.iloc[1]['rate']*100:.1f} percent; "
      f"{race_tab.iloc[2]['officer_race_f']}: {race_tab.iloc[2]['rate']*100:.1f} percent).")
print("After adjusting for borough, location type, year, and FADO type in a logistic model, "
      "officer-race coefficients are not statistically significant, while male officers show modestly higher odds of substantiation.")
print("Stratified checks by FADO type and by period show the same qualitative pattern. "
      "A sensitivity analysis excluding 'Unable to Determine' leaves the headline conclusions unchanged.")



--- Summary  ---
Among decided allegations (n=213,737), the substantiation rate is 29.1 percent.
Unadjusted, substantiation rates differ by officer race (e.g., reference ≈ White: 29.5 percent; Hispanic: 35.0 percent; Black: 30.4 percent).
After adjusting for borough, location type, year, and FADO type in a logistic model, officer-race coefficients are not statistically significant, while male officers show modestly higher odds of substantiation.
Stratified checks by FADO type and by period show the same qualitative pattern. A sensitivity analysis excluding 'Unable to Determine' leaves the headline conclusions unchanged.


In [ ]:

def fmt_pct(x): 
    return f"{100*x:.1f}%"

overall = decided["outcome_bin"].mean()
sens_overall = 0.303 

race_sorted = by_race_known.sort_values("n", ascending=False).reset_index(drop=True).copy()
gender_sorted = by_gender_known.sort_values("n", ascending=False).reset_index(drop=True).copy()

# Reference group for race disparities (largest-N)
ref_row = race_sorted.loc[0]
ref_group = ref_row["officer_race_f"]
ref_rate  = ref_row["rate"]

# Build a compact sentence for top 4 race groups
race_bits = []
for i in range(min(4, len(race_sorted))):
    g = race_sorted.loc[i, "officer_race_f"]
    r = race_sorted.loc[i, "rate"]
    diff = r - ref_rate
    sign = "+" if diff >= 0 else "−"
    race_bits.append(f"{g} {fmt_pct(r)} ({sign}{abs(diff)*100:.1f} pp vs {ref_group})")
race_sentence = "; ".join(race_bits)

# Gender sentence for the main two groups if present
gender_bits = []
for g in ["Male", "Female"]:
    row = gender_sorted[gender_sorted["officer_gender_f"] == g]
    if len(row):
        gender_bits.append(f"{g} {fmt_pct(row.iloc[0]['rate'])}")
gender_sentence = "; ".join(gender_bits) if gender_bits else "Gender rates printed above"

print("=== CCRB fairness audit summary ===")
print(f"Among decided allegations (n={len(decided):,}), the substantiation rate is {fmt_pct(overall)}.")
print(f"Sensitivity excluding 'Unable to Determine' yields {fmt_pct(sens_overall)}.")
print(f"Unadjusted by officer race: {race_sentence}.")
print(f"Unadjusted by officer gender: {gender_sentence}.")
print("Adjusted logistic regression with controls (borough, location type, year, FADO) shows no statistically significant officer-race effects,")
print("while male officers have modestly higher odds of substantiation. Context terms are strong.")
print("Interpretation per Equal Opportunity: report raw TPR gaps and the adjusted view, stratify by FADO and period,")
print("and keep calibration-by-group and EO monitoring as guardrails.")


=== CCRB fairness audit summary ===
Among decided allegations (n=213,737), the substantiation rate is 29.1%.
Sensitivity excluding 'Unable to Determine' yields 30.3%.
Unadjusted by officer race: White 29.5% (+0.0 pp vs White); Hispanic 35.0% (+5.6 pp vs White); Black 30.4% (+0.9 pp vs White); Asian 38.4% (+9.0 pp vs White).
Unadjusted by officer gender: Male 31.6%; Female 31.5%.
Adjusted logistic regression with controls (borough, location type, year, FADO) shows no statistically significant officer-race effects,
while male officers have modestly higher odds of substantiation. Context terms are strong.
Interpretation per Equal Opportunity: report raw TPR gaps and the adjusted view, stratify by FADO and period,
and keep calibration-by-group and EO monitoring as guardrails.


In [22]:

#  Penalties fairness audit
# Scope: only substantiated allegations
# Goal: check any-penalty rate and penalty severity across officer race and gender,
#       and fit a clean logistic model that is not distorted by "Unknown" coverage.


import pandas as pd
import numpy as np
import patsy as pt
import statsmodels.api as sm
from statsmodels.stats.proportion import proportion_confint

# ---- 12.0 Guardrails: make sure we have the input frames ----
# Expected: decided (from earlier steps), penalties, officers, allegations, complaints
# decided must contain outcome_bin, officer_race, officer_gender, Complaint Id, Tax ID
assert "outcome_bin" in decided.columns, "decided must have outcome_bin"
assert penalties is not None, "penalties DataFrame must be loaded"

# ---- 12.1 Keep only substantiated rows (this is our post-decision population) ----
subst = decided[decided["outcome_bin"] == 1].copy()
print(f"Substantiated allegations: {len(subst):,}")

# ---- 12.2 Choose penalty columns and join to substantiated rows ----
# Note: names match the NYC Open Data schema at time of download
pen_keep = [
    "Complaint Id", "Tax ID",
    "NYPD Officer Penalty",
    "APU CCRB Trial Recommended Penalty",
    "APU Plea Agreed Penalty",
    "APU Trial Commissioner Recommended Penalty",
    "Non-APU NYPD Penalty Report Date",
]
pen = penalties[[c for c in pen_keep if c in penalties.columns]].copy()

subst_pen = (subst.merge(pen, on=["Complaint Id","Tax ID"], how="left")
                  .rename(columns={"NYPD Officer Penalty": "final_penalty"}))

print("With any penalty field present:", subst_pen["final_penalty"].notna().sum())

# ---- 12.3 Coalesce a single penalty_text column for display and analysis ----
# Rationale: sometimes the NYPD final penalty is empty but APU recommendations exist.
def coalesce_penalty(row):
    for c in ["final_penalty",
              "APU Trial Commissioner Recommended Penalty",
              "APU Plea Agreed Penalty",
              "APU CCRB Trial Recommended Penalty"]:
        if c in row and pd.notna(row[c]) and str(row[c]).strip():
            return str(row[c]).strip()
    return np.nan

subst_pen["penalty_text"] = subst_pen.apply(coalesce_penalty, axis=1)

# Binary indicator: any penalty recorded vs none recorded
subst_pen["any_penalty"] = subst_pen["penalty_text"].notna().astype(int)

# ---- 12.4 Simple ordered severity score for quick comparisons ----
# This is a toy but monotone scale; higher means harsher.
severity_order = [
    "Instructions",
    "Formalized Training",
    "Command Level Instructions",
    "Command Discipline A",
    "Command Discipline B",
    "Charges",
    "Termination",
]
sev_map = {name.lower(): i+1 for i, name in enumerate(severity_order)}

def severity_score(s):
    if pd.isna(s):
        return 0
    key = str(s).strip().lower()
    # exact or fuzzy contains
    if key in sev_map:
        return sev_map[key]
    for name, rank in sev_map.items():
        if name in key:
            return rank
    return 0

subst_pen["penalty_severity"] = subst_pen["penalty_text"].map(severity_score)

# Fill friendly buckets for protected attributes
subst_pen["officer_race_f"]   = subst_pen["officer_race"].fillna("Unknown")
subst_pen["officer_gender_f"] = subst_pen["officer_gender"].fillna("Unknown")

# ---- 12.5 Any-penalty rates with Wilson 95% CIs by race and gender ----
def rates_with_ci(frame: pd.DataFrame, group_col: str) -> pd.DataFrame:
    t = (frame.groupby(group_col, dropna=False)["any_penalty"]
         .agg(rate="mean", n="count")
         .sort_values("n", ascending=False)
         .reset_index())
    lo, hi = proportion_confint((t["rate"]*t["n"]).round().astype(int),
                                t["n"], method="wilson")
    t["ci_low"] = lo; t["ci_high"] = hi
    return t

print("\nAny-penalty rate by officer race (substantiated):")
print(rates_with_ci(subst_pen, "officer_race_f"))

print("\nAny-penalty rate by officer gender (substantiated):")
print(rates_with_ci(subst_pen, "officer_gender_f"))

# ---- 12.6 Severity summaries by group ----
def severity_summary(frame, group_col):
    agg = (frame.groupby(group_col, dropna=False)
           .agg(n=("penalty_severity","size"),
                mean_sev=("penalty_severity","mean"),
                p_any=("any_penalty","mean"))
           .reset_index()
           .sort_values("n", ascending=False))
    return agg

print("\nPenalty severity (mean) by officer race:")
print(severity_summary(subst_pen, "officer_race_f"))

print("\nPenalty severity (mean) by officer gender:")
print(severity_summary(subst_pen, "officer_gender_f"))

# ---- 12.7 Build clean covariates for modeling any_penalty ----
# We need context variables to avoid confusing data coverage with fairness signals.
KEY = "Complaint Id"
need_cols = ["Borough Of Incident Occurrence", "Location Type Of Incident", "Incident Date"]
for c in need_cols:
    if c not in subst_pen.columns and c in complaints.columns:
        subst_pen = subst_pen.merge(complaints[[KEY, c]], on=KEY, how="left")

# Year and FADO Type
subst_pen["year"] = pd.to_datetime(subst_pen["Incident Date"], errors="coerce").dt.year
if "FADO Type" not in subst_pen.columns and "FADO Type" in allegations.columns:
    subst_pen = subst_pen.merge(
        allegations[[KEY, "FADO Type"]].drop_duplicates(), on=KEY, how="left"
    )

# ---- 12.8 Cleaned logistic model for any_penalty
# Student note: "Unknown" buckets reflect missingness, not true zero punishment.
# We model only rows where a penalty decision could be observed.
dfp = subst_pen[
    subst_pen["officer_race"].notna() &
    subst_pen["officer_gender"].notna()
].copy()

# Optional stricter filter: only cases where a penalty could reasonably be recorded.
# If your data has time-related coverage issues, uncomment the following line:
# dfp = dfp[(dfp["year"] >= 2014) | dfp["penalty_text"].notna()].copy()

terms = []
if "officer_race" in dfp.columns:
    terms.append("C(officer_race)")
if "officer_gender" in dfp.columns:
    terms.append("C(officer_gender)")
if "Borough Of Incident Occurrence" in dfp.columns:
    terms.append("C(Q('Borough Of Incident Occurrence'))")
if "Location Type Of Incident" in dfp.columns:
    terms.append("C(Q('Location Type Of Incident'))")
if "year" in dfp.columns:
    terms.append("C(year)")
if "FADO Type" in dfp.columns:
    terms.append("C(Q('FADO Type'))")

if len(terms) >= 2:  # need at least protected + one control to be useful
    formula = "any_penalty ~ " + " + ".join(terms)
    y, X = pt.dmatrices(formula, data=dfp, return_type="dataframe")
    try:
        logit_pen_clean = sm.Logit(y, X).fit(disp=False)
        print("\n[Clean fit] Logit for any_penalty:")
        print(logit_pen_clean.summary2().tables[1].head(30))
    except Exception as e:
        print("Clean logit failed (separation or sparse cells):", e)
else:
    print("Not enough covariates to fit a clean penalty model.")

# ---- 12.9 Optional: severity model for rows that received a penalty ----
# Student note: separates the question "was there any penalty" from "how harsh was it".
with_pen = subst_pen[subst_pen["penalty_severity"] > 0].copy()
if len(with_pen) > 100:  # avoid tiny-sample modeling
    import statsmodels.formula.api as smf
    sev_terms = []
    if "officer_race" in with_pen.columns:
        sev_terms.append("C(officer_race)")
    if "officer_gender" in with_pen.columns:
        sev_terms.append("C(officer_gender)")
    if "Borough Of Incident Occurrence" in with_pen.columns:
        sev_terms.append("C(Q('Borough Of Incident Occurrence'))")
    if "Location Type Of Incident" in with_pen.columns:
        sev_terms.append("C(Q('Location Type Of Incident'))")
    if "year" in with_pen.columns:
        sev_terms.append("C(year)")
    if "FADO Type" in with_pen.columns:
        sev_terms.append("C(Q('FADO Type'))")

    if sev_terms:
        sev_formula = "penalty_severity ~ " + " + ".join(sev_terms)
        sev_ols = smf.ols(sev_formula, data=with_pen).fit()
        print("\nOLS for penalty severity among those with a penalty:")
        print(sev_ols.summary())
    else:
        print("No covariates available for severity model.")
else:
    print("Too few rows with non-zero severity to model reliably.")


Substantiated allegations: 62,269
With any penalty field present: 37564

Any-penalty rate by officer race (substantiated):
    officer_race_f      rate      n        ci_low   ci_high
0            White  0.662991  28115  6.574441e-01  0.668494
1         Hispanic  0.671515  17943  6.646072e-01  0.678350
2            Black  0.681055   8155  6.708559e-01  0.691083
3          Unknown  0.000000   4468  5.421011e-20  0.000859
4            Asian  0.654530   3543  6.387124e-01  0.670013
5  American Indian  0.822222     45  6.867018e-01  0.907056

Any-penalty rate by officer gender (substantiated):
  officer_gender_f      rate      n    ci_low   ci_high
0             Male  0.670807  52817  0.666787  0.674802
1           Female  0.635149   4939  0.621623  0.648464
2          Unknown  0.002897   4487  0.001694  0.004951
3     TGNC / Other  0.730769     26  0.539170  0.862956

Penalty severity (mean) by officer race:
    officer_race_f      n  mean_sev     p_any
5            White  28115  0.360733 

c:\Users\DELL\anaconda3\envs\souq_e_commerce_3_11\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "



[Clean fit] Logit for any_penalty:
                                                        Coef.       Std.Err.  \
Intercept                                          -25.070243  124867.110183   
C(officer_race)[T.Asian]                            -0.909029       0.406305   
C(officer_race)[T.Black]                            -0.827664       0.405419   
C(officer_race)[T.Hispanic]                         -0.856330       0.405014   
C(officer_race)[T.White]                            -0.900294       0.404772   
C(officer_gender)[T.Male]                            0.217459       0.032862   
C(officer_gender)[T.TGNC / Other]                    0.224594       0.452614   
C(Q('Borough Of Incident Occurrence'))[T.Brooklyn]   0.013504       0.024193   
C(Q('Borough Of Incident Occurrence'))[T.Manhat...   0.093081       0.028060   
C(Q('Borough Of Incident Occurrence'))[T.Outsid...  -0.225199       0.240070   
C(Q('Borough Of Incident Occurrence'))[T.Queens]     0.042569       0.031662   
C(Q(

Good thing is that Race and Gender have very few missing values combined And I dont think any imputations are needed.

Wait, we cannot directly use the R code. In R those numbers were factor level indexes, not stable IDs. Factor orders can change between downloads. 